##  Part 4: Faster, more efficient data processing

Let's combine Python and Julia:

In [ ]:
from juliacall import Main as jl

In [ ]:
%load_ext juliacall

In [ ]:
%%julia

using Pkg
Pkg.add("UnROOT")
using UnROOT
using AwkwardArray

In [ ]:
tree = jl.Main.ROOTFile("./data/SMHiggsToZZTo4L.root")

In [ ]:
tree

In [ ]:
events = jl.Main.LazyTree(tree, "Events")

In [ ]:
events

In [ ]:
jl.events = events

In [ ]:
%%julia

first(events.Muon_charge, 4)

In [ ]:
%%julia

using LorentzVectorHEP
one_event = first(events);

# destruct syntax sugar
(; Muon_pt, Muon_eta, Muon_phi, Muon_mass) = one_event

muons = LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass)


In [ ]:
%%julia

function main_looper(events)
    array = AwkwardArray.PrimitiveArray{Float64}()
    for evt in events

        (; Muon_charge) = evt
        if length(Muon_charge) != 4
            continue
        end
        sum(Muon_charge) != 0 && continue # shortcut if-else
        
        (; Muon_pt, Muon_eta, Muon_phi, Muon_mass) = evt
        higgs_4vector = sum(LorentzVectorCyl.(Muon_pt, Muon_eta, Muon_phi, Muon_mass))
        higgs_mass = mass(higgs_4vector)
        
        push!(array, higgs_mass)
    end
    
    return array
end

In [ ]:
%%julia

array = @time main_looper(events)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(jl.array, bins=100, range=(90, 140));